In [19]:
pwd

'/home/gyubin/Knowledge_Editing_Dataset/analysis/Preprocessing'

In [20]:
import pandas as pd
import re


df = pd.read_excel("../../data/one_hop/raw_df.xlsx")
df.fillna('null', inplace = True)
new_df = df.copy()

In [21]:
count = 0
for i in range(len(df)):
    count += df.loc[i, 'sbj_hop_num']
    count += df.loc[i, 'obj_true_hop_num']
    count += df.loc[i, 'obj_new_hop_num']
print(count)

13208725


In [22]:
df.head()

,index,subject,subject_id,sbj_one_hop,sbj_hop_num,obj_true,obj_true_id,obj_one_hop,obj_true_hop_num,obj_new,obj_new_id,obj_new_one_hop,obj_new_hop_num,view
0,0,Danielle Darrieux,Q234149,"Darrieux,Obálky knih,ola2002159092,Q18040452,4...",175,French,Q150,"Province of Namur,present tense,history of Fre...",259,English,Q1860,"Malta,Bahrain,Vanuatu,Israel,Sierra Leone,Unit...",371,222931
1,1,Edwin of Northumbria,Q348955,"Æthelburh of Kent,http://commons.wikimedia.org...",73,Christianity,Q5043,"outline of Christianity,Portal:Christianity,mo...",140,Islam,Q432,"zakāt,Arabian mythology,Battle of Badr,Muslim ...",236,177019
2,2,Toko Yasuda,Q7813654,"rock music,Blonde Redhead,http://commons.wikim...",25,guitar,Q6607,"mahogany wood,Instrument of the Year,http://co...",115,piano,Q5994,http://commons.wikimedia.org/wiki/Special:File...,86,74330
3,3,Autonomous University of Madrid,Q788091,http://commons.wikimedia.org/wiki/Special:File...,80,Spain,Q29,"Supreme Court of Spain,Romani people,Valencian...",620,Sweden,Q34,"Universal Postal Union,Organization for Securi...",586,87615
4,4,Lyon,Q456,"La Croix-Rousse,Paulys Realenzyklopädie der kl...",259,Beirut,Q3820,"1973 Israeli raid on Lebanon,Siege of Beirut,E...",172,Manila,Q1461,"Caloocan,Quezon City,megacity,Honey Lacuna,htt...",241,2376138


In [23]:
from tqdm import tqdm

def remain_english(words):
    hop_list = []
    for word in words:
        if re.search(r'^[a-zA-Z\s]+$', word):
            hop_list.append(word)
    return hop_list
    
def remove_variations(word, words_list):
    normalized_parts = set(word.lower().split())
    words_list = [w for w in words_list if not any(part in w.lower() for part in normalized_parts)]
    return words_list

def remove_redundancy(words):
    return list(set(words))

def prep(word, words):
    one_hop = remain_english(words)
    one_hop = remove_variations(word, one_hop)
    return one_hop

def remove_elements(a, b):
    ans = [item for item in a if item not in b]
    return  ','.join(ans)


for i in tqdm(range(len(df))):
    sbj = df.loc[i, 'subject']
    sbj_words = df.loc[i, 'sbj_one_hop'].split(',')
    
    obj_true = df.loc[i, 'obj_true']
    obj_true_words = df.loc[i, 'obj_one_hop'].split(',')
    
    obj_new = df.loc[i, 'obj_new']
    obj_new_words = df.loc[i, 'obj_new_one_hop'].split(',')
    
    nosbj = [sbj] + [obj_true] + [obj_new]
    noobj = [sbj] + [obj_true] + [obj_new]
    noobjnew = [sbj] + [obj_true] + [obj_new]
    
    sbj_words = prep(sbj, sbj_words)
    obj_true_words = prep(obj_true, obj_true_words)
    obj_new_words = prep(obj_new, obj_new_words)
    
    sbj_words = remove_redundancy(sbj_words)
    obj_true_words = remove_redundancy(obj_true_words)
    obj_new_words = remove_redundancy(obj_new_words)
    
    sbj_words = remove_elements(sbj_words, nosbj)
    obj_true_words = remove_elements(obj_true_words, noobj)
    obj_new_words = remove_elements(obj_new_words, noobjnew)
    

    
    df.loc[i, 'sbj_one_hop'] = sbj_words
    df.loc[i, 'obj_one_hop'] = obj_true_words
    df.loc[i, 'obj_new_one_hop'] = obj_new_words

100%|██████████| 21919/21919 [00:19<00:00, 1142.40it/s]


In [24]:
df.head()

,index,subject,subject_id,sbj_one_hop,sbj_hop_num,obj_true,obj_true_id,obj_one_hop,obj_true_hop_num,obj_new,obj_new_id,obj_new_one_hop,obj_new_hop_num,view
0,0,Danielle Darrieux,Q234149,"Georges Mitsinkides,stage actor,singer,Henri D...",175,French,Q150,"Madagascar,synthetic language,Canada,Haiti,Cam...",259,English,Q1860,"voiced alveolar approximant,Grenada,Ethiopia,S...",371,222931
1,1,Edwin of Northumbria,Q348955,"English,saint,Hatfield Chase,Whitby Abbey,Germ...",73,Christianity,Q5043,"Bible,Christentum,Jesus,christentum,Small Broc...",140,Islam,Q432,"hadith,Marifa,idea,Ancient Semitic religion,Ne...",236,177019
2,2,Toko Yasuda,Q7813654,"Japan,Touch and Go Records,bass guitar,keyboar...",25,guitar,Q6607,"Instrument of the Year,neck,fretboard,mahogany...",115,piano,Q5994,"Bartolomeo Cristofori,kpf,Metropolitan Museum ...",86,74330
3,3,Autonomous University of Madrid,Q788091,European Alliance for Social Sciences and Huma...,80,Spain,Q29,"Cortes Generales,Extremaduran,spanyol,African ...",620,Sweden,Q34,"Scandinavian studies,svensk,African Developmen...",586,87615
4,4,Lyon,Q456,"International Cities of Refuge Network,Aleppo,...",259,Beirut,Q3820,"Athens,Yerevan,Rmeil,Bachoura,Mousseitbeh,Smal...",172,Manila,Q1461,"Navotas,Panama City,Cartagena de Indias,Ho Chi...",241,2376138


In [25]:
for i in range(len(df)):
    df.loc[i, 'sbj_hop_num'] = len(df.loc[i, 'sbj_one_hop'].split(','))
    df.loc[i, 'obj_true_hop_num'] = len(df.loc[i, 'obj_one_hop'].split(','))
    df.loc[i, 'obj_new_hop_num'] = len(df.loc[i, 'obj_new_one_hop'].split(','))

In [26]:
for i in range(len(df)):
    if df.loc[i, 'sbj_one_hop'] == '':
        df.loc[i, 'sbj_one_hop'] = 'null'
        df.loc[i, 'sbj_hop_num'] = 0
    if df.loc[i, 'obj_one_hop'] == '':
        df.loc[i, 'obj_one_hop'] = 'null'
        df.loc[i, 'obj_true_hop_num'] = 0
    if df.loc[i, 'obj_new_one_hop'] == '':
        df.loc[i, 'obj_new_one_hop'] = 'null'
        df.loc[i, 'obj_new_hop_num'] = 0

In [27]:
df.head()

,index,subject,subject_id,sbj_one_hop,sbj_hop_num,obj_true,obj_true_id,obj_one_hop,obj_true_hop_num,obj_new,obj_new_id,obj_new_one_hop,obj_new_hop_num,view
0,0,Danielle Darrieux,Q234149,"Georges Mitsinkides,stage actor,singer,Henri D...",21,French,Q150,"Madagascar,synthetic language,Canada,Haiti,Cam...",99,English,Q1860,"voiced alveolar approximant,Grenada,Ethiopia,S...",175,222931
1,1,Edwin of Northumbria,Q348955,"English,saint,Hatfield Chase,Whitby Abbey,Germ...",12,Christianity,Q5043,"Bible,Christentum,Jesus,christentum,Small Broc...",25,Islam,Q432,"hadith,Marifa,idea,Ancient Semitic religion,Ne...",76,177019
2,2,Toko Yasuda,Q7813654,"Japan,Touch and Go Records,bass guitar,keyboar...",13,guitar,Q6607,"Instrument of the Year,neck,fretboard,mahogany...",32,piano,Q5994,"Bartolomeo Cristofori,kpf,Metropolitan Museum ...",11,74330
3,3,Autonomous University of Madrid,Q788091,European Alliance for Social Sciences and Huma...,9,Spain,Q29,"Cortes Generales,Extremaduran,spanyol,African ...",231,Sweden,Q34,"Scandinavian studies,svensk,African Developmen...",212,87615
4,4,Lyon,Q456,"International Cities of Refuge Network,Aleppo,...",47,Beirut,Q3820,"Athens,Yerevan,Rmeil,Bachoura,Mousseitbeh,Smal...",51,Manila,Q1461,"Navotas,Panama City,Cartagena de Indias,Ho Chi...",93,2376138


In [28]:
count = 0
for i in range(len(df)):
    count += df.loc[i, 'sbj_hop_num']
    count += df.loc[i, 'obj_true_hop_num']
    count += df.loc[i, 'obj_new_hop_num']
print(count)

4385737


In [29]:
df.to_excel("preprocessed_df2.xlsx", index = False)